In [1]:
###Requirement Library and tools to install
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score
from sklearn.datasets import make_blobs
import folium
import seaborn as sns; sns.set()
from scipy.spatial.distance import euclidean
import csv 
import webbrowser  
import warnings
warnings.filterwarnings("ignore") 
totalBTS=3006 ###Number Of Total BTS That readed from file
knum=200 ### Initialization Number Of Cluster
k=[knum]
###Colors for each cluster 
cols = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4',
        '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', 
        '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', 
        '#000075', '#808080']*500

In [2]:

### In "BaseData/Start/" path, there are several files that correspond to the number of set BTSs, 
### The necessary files have been created for the number of 20, 200, 300 and 3006 BTS. 
### If you want to test another desired number of BTS, you can edit the files related to 3006 BTS and create Data (02Adjacency_Matrix)-X.csv and Data (01UniqueBts)-X.csv files 
### according to the naming done.

### the matrix stores the shortest paths in the df DataFrame.
df = pd.read_csv('BaseData/Start/Data (02Adjacency_Matrix)-'+str(totalBTS)+'.csv' , header=None)
###Data (01UniqueBts)-X is the bts list with bts code, user requests, lat and long
df2 = pd.read_csv('BaseData/Start/Data (01UniqueBts)-'+str(totalBTS)+'.csv' , names=["BCode", "TUsers", "WLoad", "Lat", "Long"]) 
###Generate random data between 8000 and 16000 as acceptable workload for servers
#servers=[(x+800)*10 for x in np.random.choice(range(800), knum, replace=False)]
#servers.sort()
###Random data is generated once and stored in HServers.csv file for use by other algorithms
servers = pd.read_csv('BaseData/Start/HServers.csv').to_numpy() 
### create_map function is for generate and show map with bts location and clusters
def create_map(df, cluster_column,lat,long):
    m = folium.Map(location=[df[lat].mean(), df[long].mean()], zoom_start=14) 
    for i,item in df.groupby(cluster_column):
        minlat = item[lat].min()
        minlong = item[long].min()
        maxlat = item[lat].max()
        maxlong = item[long].max()
        points = [
            [[minlat,minlong], [minlat, maxlong]],
            [[minlat,minlong], [maxlat, minlong]],
            [[maxlat,maxlong], [minlat, maxlong]],
            [[maxlat,maxlong], [maxlat, minlong]]
            
            ]
        folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(m) 
        ww=df[df[cluster_column] == i]["WLoad"].sum()
        folium.map.Marker( 
            [(minlat-0.0012), (minlong-0.0012)],
            icon=folium.DivIcon(icon_size=(110,50),html=f"""<div style="line-height:28px;font-size:38px;border-radius:10px;;font-weight:bold;;text-align:center;font-family: courier new;background-color:white; color:black">C:{str(i)+'<br/>W:'+str(ww)}</div>""")
            ).add_to(m)
    for _, row in df.iterrows():
        if row[cluster_column] == -1:
            cluster_colour = '#000000'
        else:
            cluster_colour = cols[(int)(row[cluster_column])] 
        folium.CircleMarker(
            location= [row[lat], row[long]],
            radius=5,
            popup= str(int(row[0]))+'<br/>'+str(int(row['WLoad'])),
            color=cluster_colour,
            stroke = True,
            weight=1,
            fill_opacity=1,
            opacity=1,
            fill=True,
            fill_color=cluster_colour
        ).add_to(m) 
        folium.CircleMarker(
            location= [row[lat], row[long]],
            radius=9,
            popup= str(int(row[0]))+'<br/>'+str(int(row['WLoad'])),
            color='#000000',
            stroke = True,
            fill=True,
            fill_color='#000000'
        ).add_to(m) 
    return m

In [3]:
###This function is to find the nth item in an array
def nthMin(r,n):
    row = r.copy()
    index=0  
    min=9999999
    for i in range(1,n+1): 
        min = np.max(row)
        index = np.argmax(row)
        for j in range(0,len(row)):
            if row[j] <= min and row[j] != -1:
                min = row[j]
                index = j
            #print(row)  
        if index<len(row):
            row[index]=-1  
    return index, min     

###Calculation of workload and distance of BTS and server
def BTS_Server_Di_Wl(df,df2,serverList ):  
    serverList_ = ([i - 1 for i in serverList])  
    serverList_.sort()
    selected_dfM  = df.iloc[:,serverList_]   
    newdf = df2.copy()
    newdf["SCode"]=(selected_dfM.idxmin(axis=1).to_numpy()+1).tolist()
    newdf["Distance"]=selected_dfM.min(axis=1).to_numpy().tolist()
    return newdf 

In [4]:
###This function calculates the median of a group of BTS based on their number and distance based on the formula used
def CenterOfPoints(coords,lat="lat",long="long"):
    lata=coords[[lat]].to_numpy()
    longa=coords[[long]].to_numpy()
    return [sum(lata)/len(lata),sum(longa)/len(longa)]
 
####Calculation of the workload of different sides of a middle point
def splitedWorkLoad(dfr):
    centerLat,centerLong = CenterOfPoints(dfr,'Lat','Long')
    top=dfr[dfr["Lat"] >= float(centerLat)]["WLoad"].sum()
    down=dfr[dfr["Lat"] < float(centerLat)]["WLoad"].sum()
    right=dfr[dfr["Long"] >= float(centerLong)]["WLoad"].sum()
    left=dfr[dfr["Long"] < float(centerLong)]["WLoad"].sum()
    return top,down,right,left
 ####This function specifies whether to split the current area longitudinally or transversely
def AreaSplitType(dfr): ####break horizontal:1  #break Vertical:2  #break is Same=0
    h= dfr["Lat"].max()-dfr["Lat"].min()
    w = dfr["Long"].max()-dfr["Long"].min()
    if h>w:
            return '-'
    else:
        return '|'


In [5]:
 
#### In this function, network traffic, standard deviation and distance are calculated
def Calculate(ii,file):
    df = pd.read_csv('BaseData/Data ('+file+'_List).csv')   
    dfs=df.groupby(['SCode'])['WLoad'].sum()
    nettraffic=0
    sculsterWL=[]
    for i,item in df.groupby(['SCode']):
        sculsterWL.append(item['WLoad'].sum())
        for index, row in item.iterrows():
            nettraffic+=(row[2]*row[6])
    dfs.to_csv ('BaseData/Data ('+file+'_ListServer).csv', index=None, header = True)  
    cnt = len(sculsterWL)  
    avg= sum(sculsterWL)/len(sculsterWL)
    sm=0
    for i in sculsterWL:
        sm+=(i-avg)**2
    mystd = np.sqrt(sm/cnt)
    SWorkLoad=np.std(dfs)  
    with open('BaseData/Data ('+file+'_Result).csv',  'a') as f:
        f.writelines(','.join([str(x) for x in [ii,df['BCode'].count(),int(nettraffic),(int)(mystd),(df['Distance'].sum())]])+'\n')

In [6]:
 
####This function creates a new ID for each new cluster
globalCluster = [x for x in range(1,10*knum)]
def getNewCluster():
    nval=0
    for i in range(0,10*knum-1):
        if globalCluster[i] == 0:
            continue;
        else:
            nval=globalCluster[i]
            globalCluster[i]=0
            return nval

getNewCluster()
gval=1
gvalCluster=1
####This function recursively creates clustering. The callback continues until the cluster workload exceeds the threshold
def SplitDataFrameSimple(dfr ,crrentCluster,k,avgWl,splitT): #names=["id", "users", "wl", "lat", "long", "cluster"] 
    if len(dfr[dfr["cluster"] == crrentCluster].index) == 0:
        return
    newCluster = getNewCluster()
    centerLat,centerLong = CenterOfPoints(dfr[dfr["cluster"] == crrentCluster],'Lat','Long')
    splitType = AreaSplitType(dfr[dfr["cluster"] == crrentCluster])
    topwl,downwl,rightwl,leftwl = splitedWorkLoad(dfr[dfr["cluster"] == crrentCluster])  
    global gval
    global gvalCluster
    gval=gval+1
    TreshHold=avgWl*splitT 
    if splitType == '-': 
        dfr.loc[(dfr["cluster"] == crrentCluster) & (dfr["Lat"] < float(centerLat)),"cluster"] = newCluster
        gvalCluster+=1
        #m = create_map(dfr, "cluster",'Lat','Long')
        #m.save("maps\\c"+str(crrentCluster)+"-"+str(gvalCluster)+".html") 
        if topwl > TreshHold:
            SplitDataFrameSimple(dfr,crrentCluster,k,avgWl,splitT)
        if downwl > TreshHold:
            SplitDataFrameSimple(dfr,newCluster,k,avgWl,splitT)
    else: 
        dfr.loc[(dfr["cluster"] == crrentCluster) & (dfr["Long"] < float(centerLong)),"cluster"] = newCluster
        gvalCluster+=1
        #m = create_map(dfr, "cluster",'Lat','Long')
        #m.save("maps\\c"+str(crrentCluster)+"-"+str(gvalCluster)+".html")
        if rightwl > TreshHold:
            SplitDataFrameSimple(dfr,crrentCluster,k,avgWl,splitT)
        if leftwl > TreshHold:
            SplitDataFrameSimple(dfr,newCluster,k,avgWl,splitT)
    return dfr   
 

In [7]:
####This function calculates the standard deviation for the case where the servers are heterogeneous
def Calc_SD(WLoads,S_Proc): 
    ii=len(WLoads)
    W_hat = 0 
    SD=0
    for k in range(0,ii):
        W_hat+=WLoads[k]/S_Proc[k]; 
    W_hat=W_hat/ii; 
    for k in range(0,ii):
        SD+=(WLoads[k]/S_Proc[k]-W_hat)**2;  
   
    return np.sqrt((SD/ii))

In [8]:
#### The main function that first calls the recursive clustering function and then updates the deployment of BTS according to the status of the clusters according to the number of episodes we have considered
splitT = 1.6
def RecursiveClustering(df,df2,k,file,episode):  
    df2["cluster"]=1 
    avgWl=(df2["WLoad"].sum()/k)  
    
    #m = create_map(df2, "cluster",'Lat','Long')************************************
    #m.save("maps\\c0-1.html")******************************************************
    SplitDataFrameSimple (df2,1,k,avgWl,splitT) 
    df2["SCode"]=-1
    df3=df2.copy()
    while episode>0: 
        addedServer=[]  
        for i,item in df3.groupby("cluster"):
            cdf = df.filter(items = [x-1 for x in (item["BCode"]).values.tolist()]  , axis=0)
            scode=-1
            minDW = item["WLoad"].max()*999999 
            for index,row in cdf.iterrows():
                btsdi=0
                btsdi= np.sum([row2[index]*int(df3.loc[(df3["BCode"] == index2+1),"WLoad"]) for index2,row2 in cdf.iterrows()])
                if btsdi< minDW:
                    minDW = btsdi
                    scode = index+1 
            addedServer.append(int(scode))   
           
        df3 =BTS_Server_Di_Wl(df,df3,addedServer ) 
        df3["cluster"] = df3["SCode"]
        temp =(df3.groupby("cluster", sort=True)["WLoad"].sum().reset_index()  )
        temp2=(temp.sort_values(by = ['WLoad'], ascending=[True])).reset_index() 
        
        df_HServers = pd.read_csv('BaseData/Start/HServers3.csv').to_numpy() 

        sculsterWL=np.array(temp2['WLoad'])
        H_index=0
        H_WL = 0
        sculsterWL[::-1].sort() 
        print('len',len(sculsterWL), len(df_HServers))
        for i in sculsterWL: 
            H_WL = H_WL + abs(df_HServers[H_index] - i)
            H_index+=1 
        print('SD',Calc_SD(sculsterWL,df_HServers)) 
        create_map(df3, "SCode", "Lat", "Long") 
        episode-=1
    del df3["cluster"] 
    df3.to_csv ('BaseData/Data ('+file+'_List).csv', index=None, header = True)   
    return len(addedServer),df3 
####Calling the clustering function for the desired number of states
for p in range(1,2): 
    k3,df4 = RecursiveClustering(df,df2.copy(),k[0],'RecursiveClustering',p)  



len 7 95
SD [0.00025892]


In [9]:
#ُ###The display of clustering is created in such a way that the BTSs in each cluster are displayed with different colors
m=create_map(df4, "SCode", "Lat", "Long")
 
m